In [19]:
!tar -zcvf Xception-diabetic-retinopathy.tgz Xception-diabetic-retinopathy..h5

tar: Xception-diabetic-retinopathy..h5: Cannot stat: No such file or directory
tar: Exiting with failure status due to previous errors


In [20]:
pwd

'/home/wsuser/work'

In [24]:
!pip install tensorflow==2.7.1
# !pip install keras

     |████████████████████████████████| 495.2 MB 34 kB/s s eta 0:00:011████▉                           | 75.0 MB 12.8 MB/s eta 0:00:33
     |████████████████████████████████| 14.1 MB 96.6 MB/s eta 0:00:01
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.7.2
    Uninstalling tensorflow-2.7.2:
      Successfully uninstalled tensorflow-2.7.2


In [25]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
#import Lib
from tensorflow.keras.layers import Dense ,Flatten , Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator , load_img
from tensorflow.keras.applications.xception import Xception ,preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

2022-11-23 08:20:12.350142: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ibm/dsdriver/lib:/opt/oracle/lib:/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/tensorflow


In [26]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [27]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='9a95qRRGEsHPZOsJuuyYTFrmscLxBPagBWUFUL2IGcVB',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'diabeticretinopathy-donotdelete-pr-cpzkh9q8len9b8'
object_key = 'dataset.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [28]:
# Unzip the Dataset Zip File
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()), 'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [29]:
%%bash
ls dataset

testing
training


In [32]:
pwd

'/home/wsuser/work'

Train and Test

In [33]:
imageSize = [299,299]
testPath = r"/home/wsuser/work/dataset/testing"
trainPath = r"/home/wsuser/work/dataset/training"

In [34]:
#import Lib
from tensorflow.keras.layers import Dense ,Flatten , Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator , load_img
from tensorflow.keras.applications.xception import Xception ,preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

Image Data Generator

In [31]:
testData_gen = ImageDataGenerator(rescale=1./255 , 
                                  shear_range=0.2 , zoom_range=0.2 , horizontal_flip=True)
trainData_gen = ImageDataGenerator(rescale=1./255)

In [35]:
#apply image dataGenerator
testSet = testData_gen.flow_from_directory('/home/wsuser/work/dataset/testing',
                                           target_size=(299,299),
                                           batch_size=32,
                                           class_mode = 'categorical')
trainSet = trainData_gen.flow_from_directory('/home/wsuser/work/dataset/training',
                                          target_size=(299,299),
                                          batch_size=32,
                                          class_mode = 'categorical')

Found 94 images belonging to 2 classes.
Found 519 images belonging to 2 classes.


Model Building


 1.  Pre-Train CNN Model

In [36]:
xCeption = Xception(input_shape=imageSize+[3] , weights='imagenet' , include_top=False)

2022-11-23 08:24:00.262649: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ibm/dsdriver/lib:/opt/oracle/lib:/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/tensorflow
2022-11-23 08:24:00.262692: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)


83697664/83683744 [==============================] - 0s 0us/step


In [37]:
#don't train existing weights
for layer in xCeption.layers:
  layer.trainable = False

In [38]:
#our layers you can add more if you want
a = Flatten()(xCeption.output)

2. Adding Dense Layer

In [39]:
prediction = Dense(5 , activation = 'softmax')(a)
#create model object
model = Model(inputs = xCeption.input , outputs = prediction)

In [40]:
#view model structure
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 149, 149, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                             

3. Configure The Learning Process

In [41]:
#tell the model what cost and optimization method to use
model.compile(
    loss='categorical_crossentropy',
    optimizer = 'adam',
    metrics =['accuracy']
)

Train The Model

In [18]:
#fit the model
r = model.fit_generator(
    trainSet ,
    validation_data=testSet,
    epochs= 30,
    steps_per_epoch = len(trainSet)//32,
    validation_steps = len(testSet)//32
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


Epoch 1/30
3/3 [==============================] - 36s 11s/step - loss: 9.4370 - accuracy: 0.4479
Epoch 2/30
3/3 [==============================] - 28s 9s/step - loss: 10.8605 - accuracy: 0.5938
Epoch 3/30
3/3 [==============================] - 30s 9s/step - loss: 7.0178 - accuracy: 0.5312
Epoch 4/30
3/3 [==============================] - 29s 9s/step - loss: 6.7019 - accuracy: 0.5104
Epoch 5/30
3/3 [==============================] - 30s 10s/step - loss: 5.5164 - accuracy: 0.7396
Epoch 6/30
3/3 [==============================] - 28s 8s/step - loss: 7.4899 - accuracy: 0.7604
Epoch 7/30
3/3 [==============================] - 28s 8s/step - loss: 4.6039 - accuracy: 0.7188
Epoch 8/30
3/3 [==============================] - 27s 8s/step - loss: 2.9316 - accuracy: 0.7396
Epoch 9/30
3/3 [==============================] - 28s 8s/step - loss: 3.8301 - accuracy: 0.6667
Epoch 10/30
3/3 [==============================] - 28s 8s/step - loss: 3.6389 - accuracy: 0.6354
Epoch 11/30
3/3 [===================

In [42]:
model.save('updated-Xception-diabetic-retinopathy.h5')

/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


In [2]:
!pip install ibm_watson_machine_learning

In [3]:
from ibm_watson_machine_learning import APIClient

wml_credentials = {
    'url':'https://us-south.ml.cloud.ibm.com',
    'apikey':'vClNzhxU8URgXTBg8u4kI-kY18QkMg6cVUTOgqAAAXAF'
}

client = APIClient(wml_credentials)

In [4]:
client

In [5]:
client.spaces.get_details()

{'resources': [{'entity': {'compute': [{'crn': 'crn:v1:bluemix:public:pm-20:us-south:a/5b2acccf8b1a4da19eb12a827e5f25ca:c9c3aa41-7761-4d42-b58c-ca5ebc35a39f::',
      'guid': 'c9c3aa41-7761-4d42-b58c-ca5ebc35a39f',
      'name': 'Watson Machine Learning-zk',
      'type': 'machine_learning'}],
    'description': '',
    'name': 'image_processing',
    'scope': {'bss_account_id': '5b2acccf8b1a4da19eb12a827e5f25ca'},
    'stage': {'production': False},
    'status': {'state': 'active'},
    'storage': {'properties': {'bucket_name': '88458888-07b9-4427-9ace-58e68637e53f',
      'bucket_region': 'us-south',
      'credentials': {'admin': {'access_key_id': '23c89aa2b0a34a288d5995069e718521',
        'api_key': 'g3llIX5sJrfxmd830UYv9mMEAMtGOB0mV2SWHoxMKi_b',
        'secret_access_key': 'a045d313e9bf2b4d3a4ee0113426c896e67b83a4678eb861',
        'service_id': 'ServiceId-17fa67d8-9e24-4623-bc44-d4c2c3b26a72'},
       'editor': {'access_key_id': '652bed11278049f3b68da2d06b87e6c2',
        'api

In [6]:
client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  ----------------  ------------------------
ID                                    NAME              CREATED
7efec339-7349-4901-99e1-a19b13790b3d  image_processing  2022-11-19T08:55:44.678Z
------------------------------------  ----------------  ------------------------


In [7]:
space_uid = "7efec339-7349-4901-99e1-a19b13790b3d"

In [8]:
client.set.default_space(space_uid)

'SUCCESS'

In [9]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [10]:
software_space_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_space_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [44]:
model_details = client.repository.store_model(model="Xception-diabetic-retinopathy.tgz", meta_props={
    client.repository.ModelMetaNames.NAME:"image_processing",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_space_uid
})

In [45]:
model_details

{'entity': {'hybrid_pipeline_software_specs': [],
  'software_spec': {'id': 'acd9c798-6974-5d2f-a657-ce06e986df4d',
   'name': 'tensorflow_rt22.1-py3.9'},
  'type': 'tensorflow_2.7'},
 'metadata': {'created_at': '2022-11-23T08:25:32.237Z',
  'id': '036e47c3-b214-4fe8-9f12-5550e516b8d9',
  'modified_at': '2022-11-23T08:25:34.379Z',
  'name': 'image_processing',
  'owner': 'IBMid-666002IBOC',
  'resource_key': 'a94ff552-8ab7-4450-b630-e5c1e5478a10',
  'space_id': '7efec339-7349-4901-99e1-a19b13790b3d'},
 'system': {'warnings': []}}

In [46]:
model_id = client.repository.get_model_id(model_details)
model_id

'036e47c3-b214-4fe8-9f12-5550e516b8d9'

In [47]:
client.repository.download(model_id, 'Diabetic_Retinopathy_Detection.tgz')

Successfully saved model content to file: 'Diabetic_Retinopathy_Detection.tgz'


'/home/wsuser/work/Diabetic_Retinopathy_Detection.tgz'